In [1]:
import numpy as np
import pandas as pd
import torch
import torchtext
import zipfile
import pathlib
from pathlib import Path
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import numericalize_tokens_from_iterator
from torch.nn.utils.rnn import pad_sequence

import logging
import os
import warnings
warnings.filterwarnings("ignore")

# Due to warning when initializing the "spacy" tokenizer
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # disable tensorflow logging
logging.getLogger('tensorflow').disabled = True  # disable tensorflow warning messages



In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:

df = pd.read_csv('artifacts/train_cleaned.csv')

# limit df to 5000
df = df[:20]
df

,tweet,sentiment,label
0,im getting on borderlands and i will murder yo...,Positive,3
1,I am coming to the borders and I will kill you...,Positive,3
2,im getting on borderlands and i will kill you ...,Positive,3
3,im coming on borderlands and i will murder you...,Positive,3
4,im getting on borderlands 2 and i will murder ...,Positive,3
5,im getting into borderlands and i can murder y...,Positive,3
6,So I spent a few hours making something for fu...,Positive,3
7,So I spent a couple of hours doing something f...,Positive,3
8,So I spent a few hours doing something for fun...,Positive,3
9,So I spent a few hours making something for fu...,Positive,3


In [8]:
df.isnull().sum()

tweet        0
sentiment    0
label        0
dtype: int64

In [4]:
df['label'].unique()

array([3, 2])

In [5]:
df_valid = pd.read_csv('artifacts/valid_cleaned.csv')

df_valid = df_valid[:500]
df_valid

,tweet,sentiment,label
0,I mentioned on Facebook that I was struggling ...,Irrelevant,0
1,BBC News - Amazon boss Jeff Bezos rejects clai...,Neutral,2
2,@Microsoft Why do I pay for WORD when it funct...,Negative,1
3,"CSGO matchmaking is so full of closet hacking,...",Negative,1
4,Now the President is slapping Americans in the...,Neutral,2
...,...,...,...
495,special shoutouts to microsoft excel 2013,Positive,3
496,Dumb Lucky☘️ (Fortnite Montage) youtu.be/psW...,Irrelevant,0
497,Dang there goes my birthday present but maybe ...,Positive,3
498,It was ab fab seeing the 6 bungalows built in ...,Irrelevant,0


In [6]:
df_valid['label'].unique()

array([0, 2, 1, 3])

## Plan of action

```
1. Write a Custom Dataset

2. collate function, that will make the data 'ready to fit' before feed the model

3. model class
```


In [20]:
import pandas as pd
import torch

from typing import Tuple

class CustomDataset(torch.utils.data.Dataset):
    """Custom dataset class for text classification."""
    
    def __init__(self,text,target):
        self.text = text
        self.target = target
        
    def __len__(self) -> int:
        """Return length of dataset."""
        return len(self.text)
    
    def __getitem__(self, index: int) -> Tuple[str, int]:
        """Return item at given index."""
       
        text = self.text[index]
        target = self.target[index]
        
        return text, target
    

    

df = pd.read_csv('artifacts/train_cleaned.csv')

# limit df to 20
df = df[:20]

train_data = CustomDataset(df['tweet'],df['label'])
train_iter = iter(train_data)

for i in train_data:
    print(i)    

('im getting on borderlands and i will murder you all ,', 3)
('I am coming to the borders and I will kill you all,', 3)
('im getting on borderlands and i will kill you all,', 3)
('im coming on borderlands and i will murder you all,', 3)
('im getting on borderlands 2 and i will murder you me all,', 3)
('im getting into borderlands and i can murder you all,', 3)
("So I spent a few hours making something for fun. . . If you don't know I am a HUGE @Borderlands fan and Maya is one of my favorite characters. So I decided to make myself a wallpaper for my PC. . Here is the original image versus the creation I made :) Enjoy! pic.twitter.com/mLsI5wf9Jg", 3)
("So I spent a couple of hours doing something for fun... If you don't know that I'm a huge @ Borderlands fan and Maya is one of my favorite characters, I decided to make a wallpaper for my PC.. Here's the original picture compared to the creation I made:) Have fun! pic.twitter.com / mLsI5wf9Jg", 3)
("So I spent a few hours doing something f

KeyError: 20

In [ ]:
train_data = CustomDataset(df['tweet'],df['label'])

In [ ]:
train_iter = iter(train_data)
train_iter

In [ ]:
next(train_iter)

In [ ]:
next(train_iter)

In [ ]:
data[0]

In [ ]:
len(df)

In [ ]:
train_data = CustomDataset(df['tweet'],df['label'])
train_iter = iter(train_data)

In [ ]:
train_data = CustomDataset(df['tweet'],df['label'])
train_iter = iter(train_data)

for i in train_iter:
    print(i)

In [22]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
import torch

from typing import Tuple

class CustomDataset(torch.utils.data.Dataset):
    """Custom dataset class for text classification."""
    
    def __init__(self,text,target):
        self.text = text
        self.target = target
        
    def __len__(self) -> int:
        """Return length of dataset."""
        return len(self.text)
    
    def __getitem__(self, index: int) -> Tuple[str, int]:
        """Return item at given index."""
       
        text = self.text[index]
        target = self.target[index]
        
        return text, target
    

    

df = pd.read_csv('artifacts/train_cleaned.csv')
df = df[:20]

train_data = CustomDataset(df['tweet'],df['label'])
  
tokenizer = get_tokenizer("spacy")

train_data = CustomDataset(df['tweet'],df['label'])
train_iter = iter(train_data)

def yield_tokens(data_iter):
    for text,_ in data_iter:
        text = text.lower()
        
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

KeyError: 20

In [25]:
df = pd.read_csv('artifacts/train_cleaned.csv')

# Check the length of the DataFrame
data_length = len(df)

# Limit the DataFrame to a maximum of 20 rows
max_rows = min(data_length, 20)  # Replace 20 with the desired length or len(df)
df = df[:max_rows]

train_data = CustomDataset(df['tweet'], df['label'])

tokenizer = get_tokenizer("spacy")

train_iter = iter(train_data)

def yield_tokens(data_iter):
    for text, _ in data_iter:
        text = text.lower()
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


KeyError: 20

In [ ]:

def text_pipeline(text):
    text = text.lower()
    tokens = tokenizer(text)
    return tokens

In [ ]:
text_pipeline(data[1][0])

In [ ]:
from torch.utils.data import DataLoader

def collate_batch(batch):
    text_list, label_list = [], []
    for (_text, _label) in batch:
        processed_text = text_pipeline(_text)  # Removed torch.tensor and dtype=torch.float
        text_list.append(processed_text)
        label_list.append(_label)
        
    text_list = torch.cat(text_list)  # Added torch.stack for concatenating tensors
    label_list = torch.tensor(label_list, dtype=torch.long)
    
    return text_list, label_list


In [ ]:
for item in data[0:5]:
    print(item)


In [ ]:
_text,_label = data[0]

processed_text = text_pipeline(_text)  # Removed torch.tensor and dtype=torch.float
text_list.append(processed_text)

label_list.append(_label)

text_list = torch.cat(text_list)  # Added torch.stack for concatenating tensors
label_list = torch.tensor(label_list, dtype=torch.long)

text_list

In [ ]:

train_iter = iter(data)

text_list, label_list = [], []
try:
    for _text, _label in train_iter:
        processed_text = text_pipeline(_text)
        text_list.append(processed_text)
        
        label_list.append(_label)

except KeyError:
    pass

label_list = torch.tensor(label_list, dtype=torch.long)


label_list

In [ ]:
type(label_list)

In [ ]:
dataloader = DataLoader(data, batch_size=5, shuffle=False, collate_fn=collate_batch)

In [ ]:
for batch,(X,y) in enumerate(dataloader):
    print('batch:',batch,)
    print(X,y)

In [ ]:
tokenizer = get_tokenizer("spacy")


def token_gen(text):
    """
    Tokenizes each sentence in a given text and yields the resulting tokens.

    Args:
        text (list[str]): A list of sentences to tokenize.

    Yields:
        list[str]: The resulting tokens from each sentence.
    """
    for sent in text:
        tokens = tokenizer(sent)
        yield tokens



In [ ]:
vocab = build_vocab_from_iterator(token_gen(df['tweet']),specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])  ## to handel OOV problem



In [ ]:
#print(vocab.get_stoi())

In [ ]:
# numericalize tokens from iterator using vocab
sequence = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df['tweet']))

print('data type is:',type(sequence))

count=0
for ids in sequence:
    print([num for num in ids])
    count+=1
    if count==10:
        break
    


In [ ]:
## check how "numericalize_tokens_from_iterator" works

from torchtext.data.functional import numericalize_tokens_from_iterator

sequence = numericalize_tokens_from_iterator(vocab,["hi how are you", "what is your name?"])
list(next(sequence))

In [ ]:
# numericalize tokens from iterator using vocab
sequence = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df['tweet']))

# create a list to store tokenized sequences
text = []
for i in range(len(df)):
    x = list(next(sequence))
    text.append(x)

# Pad the sequences to the same length along dimension 0
padded_text = pad_sequence([torch.tensor(x) for x in text], batch_first=True, padding_value=0)

MAX_LENGTH = 100
padded_text = padded_text[:,:MAX_LENGTH]

print(padded_text.shape)
print(padded_text)

In [ ]:
len(padded_text)

In [ ]:
padded_text[0]

In [ ]:
tokenizer('I will see you!')

In [ ]:
## vocab([tokens])

vocab(tokenizer('I will see you!')) ## similar to the 'fit_on_texts()' in tensorflow

In [ ]:
torch.tensor(vocab(tokenizer('I will see you')))

In [ ]:
#len(vocab.get_stoi())
len(vocab)

In [ ]:
# ?torch.nn.Embedding

In [ ]:
embedd = torch.nn.Embedding(num_embeddings=len(vocab),embedding_dim=5,padding_idx=0) 

## if we want to add embedding for a text, we should assign the value to "num_embeddings" according to the 
## max 'integer_id' that is present in the tokenized text


In [ ]:
test_input = embedd(torch.tensor(vocab(tokenizer('I will see you nonsense!'))))
test_input

In [ ]:
test_input.shape

In [ ]:
# Create the Embedding module with the correct weight matrix size
embedd = torch.nn.Embedding(len(vocab), 5, padding_idx=0)

# Check the shape of the padded_text and compare it to the expected input shape of the Embedding module
print(padded_text.shape)
# should be: torch.Size([batch_size, sequence_length])

# Use the Embedding module with the padded_text
input_text = embedd(padded_text)
print(input_text)

In [ ]:
# Check the shape of the padded_text and compare it to the expected input shape of the Embedding module
print(padded_text[0].shape)
# should be: torch.Size([batch_size, sequence_length])

# Use the Embedding module with the padded_text
embedd(padded_text[0]).shape


In [ ]:
df

In [ ]:
print(padded_text.shape)
padded_text

In [ ]:
label =df['label'].to_list()

#label

In [ ]:
label= torch.tensor(label)
print(label.shape)
label

In [1]:
len(label.unique())

NameError: name 'label' is not defined

In [ ]:
# Import required libraries
import torch.nn as nn

# Determine the number of classes
num_classes = len(label.unique())

# Define the RNNClassify module
class RNNClassify(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        
        # Define the embedding layer
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # Define the RNN layer
        self.rnn = nn.RNN(embed_dim, hidden_size,batch_first=True)
        
        # Define the linear layer
        self.linear = nn.Linear(hidden_size, num_classes)
        
        # Initialize the weights of the module
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embed.weight.data.uniform_(-initrange, initrange)
        self.rnn.weight_ih_l0.data.uniform_(-initrange, initrange)
        self.rnn.weight_hh_l0.data.uniform_(-initrange, initrange)
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        
    def forward(self, input):
        # Embed the input
        embedded = self.embed(input)
        #print('embedded shape:',embedded.shape)
        
        # Pass the embedded input through the RNN layer
        output, hidden = self.rnn(embedded)
        #print('rnn output shape:',output.shape)
        #print('rnn hidden shape:',hidden.shape)
        
        output = output[:, -1, :]  # taking last output of RNN
        #print('rnn last output shape:',output.shape)
        
        # Pass the output through the linear layer
        output = self.linear(output)
        
        # Return the output
        return output


In [ ]:
VOCAB_SIZE = len(vocab)
VOCAB_SIZE

In [ ]:
model = RNNClassify(vocab_size=VOCAB_SIZE,embed_dim=100,hidden_size=32).to(device)

In [ ]:
padded_text[0]

In [ ]:
padded_text[0].shape

In [ ]:
padded_text[0].unsqueeze(0).shape

In [ ]:
model(padded_text[0].unsqueeze(0))

In [ ]:
model(padded_text[0].unsqueeze(0)).shape

In [ ]:
padded_text

In [ ]:
# can we restrict the length of every sequence in the padded_text?

In [ ]:
padded_text.shape

In [ ]:
model(padded_text)  

In [ ]:
model(padded_text).shape            

### will try `Batch Gradient Descent`

#### first of all, let me fix `(X_train,y_train)` and `(X_test,y_test)`

In [ ]:
# X_train,y_train

X_train,y_train = padded_text,label

In [ ]:
df_valid

In [ ]:
len(df_valid)

In [ ]:
# numericalize tokens from iterator using vocab
sequence = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df_valid['tweet']))

print('data type is:',type(sequence))

count=0
for ids in sequence:
    print([num for num in ids])
    count+=1
    if count==10:
        break
    


In [ ]:
valid_token_ids = []
for i in range(len(df_valid)):
    token_id = vocab(tokenizer(df_valid['tweet'][i]))
    valid_token_ids.append(token_id)
    
    
valid_token_ids

In [ ]:
# valid_token_ids = torch.tensor(valid_token_ids) # this will throw an error, because all sequence are not of same length

# Pad the sequences to the same length along dimension 0
padded_text_valid = pad_sequence([torch.tensor(x) for x in valid_token_ids], batch_first=True, padding_value=0)
# here look, <UNK> will be assign to 0 and padding_idx will be assign also 0

padded_text_valid = padded_text_valid[:,:MAX_LENGTH]

print(padded_text_valid.shape)
print(padded_text_valid)

In [ ]:
label_valid = df_valid['label'].to_list()

In [ ]:
label_valid = torch.tensor(label_valid)
#label_valid

In [ ]:
# X_test,y_test

X_test, y_test = padded_text_valid, label_valid
X_test, y_test

#### Now, write the train and test loop for `Batch Gradient Descent`

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()  # remember it gives logits (row outputs)

In [ ]:
# Here, we will use Batch Gradient Descent, BUT, generally we prefer Mini-Batch Gradient Descent

epochs = 50


for epoch in range(epochs):
    train_loss,train_acc = 0,0
    
    # Set model to training mode
    model.train()
    
    X_train,y_train = X_train.to(device), y_train.to(device)
    
    y_logits = model(X_train)
    #print('shape of y_logits:',y_logits.shape)

    
    # Compute loss with one-hot encoded targets
    loss = loss_fn(y_logits, y_train)
    
    train_loss += loss
    train_acc += (y_logits.argmax(1) == y_train).sum().item() / len(y_train)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    
    
    with torch.inference_mode():
        
        model.eval()
        
        test_loss,test_acc = 0,0
    
        X_test, y_test = X_test.to(device), y_test.to(device)
        
        y_logits = model(X_test)

        # Compute loss with one-hot encoded targets
        loss = loss_fn(y_logits, y_test)

        test_loss += loss.item()
            
        # Compute accuracy
        test_preds = y_logits.argmax(dim=1)
        test_acc += (test_preds == y_test).sum().item() / len(y_test)

        
        print(f'Epoch {epoch+1}/{epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')
        
        print('--'*50)